In [73]:
## Import necessary modules
import os,sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, FormatStrFormatter, ScalarFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee

%matplotlib tk

In [89]:
## import data
## 1458_daily Dateien sind ab 01.03.2017 viertelstündlich
## CE_alle_agenten sind ausschliesslich stündlich
hotlinedata=pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/pkl_plots_calls_1458.pkl')
hotlinedata2=pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/doe_pickle_1458.pkl')
hl_dates=hotlinedata.date.copy() # gibt die date-spalte als eine Series aus datetime.date Daten zurück
hl_dates_sorted=hl_dates.sort_values(ascending=True).copy() # sortiert die Serie
dateindex_start=hl_dates_sorted.unique()[0] # erstes datum
dateindex_ende=hl_dates_sorted.unique()[-1] # letztes datum
einheitsindex=pd.date_range(dateindex_start, dateindex_ende)

In [91]:
hotlinedata2.head(6) # der hier hat keine halben stunden, dafuer aber schon kern-und nebenzeit
hotlinedata.head(6)

,month,year,0,1,2,3,4,5,6,7,...,21,22,23,24,summa,xlday,day,date,week,weekday
tix,11,2016,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,06:30-07:30,...,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,NaN,42688.0,14,2016-11-14,46,Mon
angekommen,11,2016,0,0,0,0,0,0,1,5,...,0,1,0,0,321,42688.0,14,2016-11-14,46,Mon
verbunden,11,2016,0,0,0,0,0,0,0,4,...,0,0,0,0,221,42688.0,14,2016-11-14,46,Mon
verloren,11,2016,0,0,0,0,0,0,1,1,...,0,1,0,0,100,42688.0,14,2016-11-14,46,Mon
servicelevel,11,2016,NaN,NaN,NaN,NaN,NaN,NaN,0,0.8,...,NaN,0,NaN,NaN,0.688474,42688.0,14,2016-11-14,46,Mon
tix,11,2016,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,06:30-07:30,...,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,NaN,42689.0,15,2016-11-15,46,Tue


In [87]:
nebenzeit=hotlinedata.loc[hotlinedata2['bz'] == 'n'].copy()
kernzeit=hotlinedata.loc[hotlinedata2['bz'] == 'k'].copy()

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [79]:
funx={'yy':'first','mm':'first','ww':'first','wd':'first','an':'sum','vb':'sum','vl':'sum'}
nebenzeit_nach_datum=nebenzeit.groupby('dt').agg(funx).copy()
kernzeit_nach_datum=kernzeit.groupby('dt').agg(funx).copy()

In [80]:
kzneu=kernzeit_nach_datum.reindex(einheitsindex, fill_value=0).drop(['ww','wd','mm','yy'],axis=1).copy() # kernzeit kriegt den index für alle vorhandenen Tage und verliert erstmal ein paar Spalten
nzneu=nebenzeit_nach_datum.reindex(einheitsindex, fill_value=0).drop(['ww','wd','mm','yy'],axis=1).copy() # same here

In [81]:
nzneu['yy']=nzneu.index.year
nzneu['mm']=nzneu.index.month
nzneu['wd']=nzneu.index.strftime('%a')
nzneu['ww']=nzneu.index.weekofyear

In [82]:
kzneu['yy']=kzneu.index.year
kzneu['mm']=kzneu.index.month
kzneu['wd']=kzneu.index.strftime('%a')
kzneu['ww']=kzneu.index.weekofyear

In [83]:
kzneu

,an,vl,vb,yy,mm,wd,ww
2016-11-14,321,100,221,2016,11,Mon,46
2016-11-15,152,20,132,2016,11,Tue,46
2016-11-16,75,11,64,2016,11,Wed,46
2016-11-17,91,7,84,2016,11,Thu,46
2016-11-18,112,22,90,2016,11,Fri,46
2016-11-19,36,9,27,2016,11,Sat,46
2016-11-20,5,0,5,2016,11,Sun,46
2016-11-21,184,25,159,2016,11,Mon,47
2016-11-22,79,15,64,2016,11,Tue,47
2016-11-23,64,8,56,2016,11,Wed,47


In [84]:
nzneu

,an,vl,vb,yy,mm,wd,ww
2016-11-14,0,0,0,2016,11,Mon,46
2016-11-15,0,0,0,2016,11,Tue,46
2016-11-16,0,0,0,2016,11,Wed,46
2016-11-17,0,0,0,2016,11,Thu,46
2016-11-18,0,0,0,2016,11,Fri,46
2016-11-19,0,0,0,2016,11,Sat,46
2016-11-20,0,0,0,2016,11,Sun,46
2016-11-21,0,0,0,2016,11,Mon,47
2016-11-22,0,0,0,2016,11,Tue,47
2016-11-23,0,0,0,2016,11,Wed,47
